In [1]:
import requests
import json
import time
import math
import numpy as np
from astropy import wcs
from astropy.io import fits
import sys
import os

__all__ = ['File']

class File:
    def __init__(self, dirtofile):
        self.dirtofile = dirtofile
        
    def tworuns(self,apikey,ra,dec,radius,scalelower,scaleupper,tweakorder,usese,crpix,parity,cutoff,processprint):
        R1 = requests.post('http://nova.astrometry.net/api/login', data={'request-json': json.dumps({"apikey": apikey})})
        if (processprint==True):
            print("LOGIN:")
            print(R1.json(),"\n")
        
        
        
        fitsfile = fits.open(self.dirtofile)
        data = fitsfile[0].data
        header = fitsfile[0].header
        data = (86.*(math.pow(2,int(header['BITPIX']))/(data-1600.))**1.4).astype(np.int16)
        if (processprint==True):
            print("transformation completed")
        
        dirtofilet = self.dirtofile[0:-5]+"_t"+self.dirtofile[-5:]
        fits.writeto(dirtofilet,data,header,overwrite=True)
        if (processprint==True):
            print("transformation file saved")
        
        
        
        R2 = requests.post('http://nova.astrometry.net/api/upload', data={'request-json': json.dumps({"session": R1.json()["session"], "allow_commercial_use": "d", "allow_modifications": "d", "publicly_visible": "y", "scale_units": "degwidth", "scale_type": "ul", "scale_lower": scalelower, "scale_upper": scaleupper, "center_ra": ra, "center_dec": dec, "radius": radius, "tweak_order": tweakorder, "use_sextractor": usese, "crpix_center": crpix, "parity": parity})}, files={'file': open(dirtofilet, 'rb')})
        if (processprint==True):
            print("\nSUBMISSION:")
            print(R2.json(),"\n")
        
        
        
        r3strs = 'http://nova.astrometry.net/api/submissions/'
        r3strm = str(R2.json()["subid"])
        R3 = requests.post(str(r3strs + r3strm))
        if (processprint==True):
            sys.stdout.write("solving ")
        counter = 0

        while len(R3.json()["job_calibrations"])==0:
            if (counter>cutoff):
                print("\nSolution failed, taking over",cutoff,"seconds\nExiting program")
                sys.exit()
            time.sleep(1)
            counter += 1
            if (processprint==True):
                sys.stdout.write(".")
            R3 = requests.post(str(r3strs + r3strm))

        time.sleep(1)
        if (processprint==True):
            print("\n\nSOLVED:")
            print(R3.json())
        
        
        
        r4strs = 'http://nova.astrometry.net/api/jobs/'
        r4strm = str(R3.json()["jobs"])[1:-1]
        r4stre = '/calibration/'
        R4 = requests.post(str(r4strs+r4strm+r4stre))
        rasol1 = R4.json()["ra"]
        decsol1 = R4.json()["dec"]



        r5str = 'http://nova.astrometry.net/new_fits_file/' + r4strm
        R5 = requests.get(r5str, allow_redirects=True)
        dirtosol1file = dirtofilet[0:-5]+"_sol1"+dirtofilet[-5:]

        filesol1 = open(str(dirtosol1file),'wb')
        filesol1.write(R5.content)
        filesol1.close()
        if (processprint==True):
            print("\nfirst solution file saved")

            print()
            print()
            print()

        
        
        if (processprint==True):
            print("SECOND RUN")
        R2 = requests.post('http://nova.astrometry.net/api/upload', data={'request-json': json.dumps({"session": R1.json()["session"], "allow_commercial_use": "d", "allow_modifications": "d", "publicly_visible": "y", "scale_units": "degwidth", "scale_type": "ul", "scale_lower": scalelower, "scale_upper": scaleupper, "center_ra": rasol1, "center_dec": decsol1, "radius": radius, "tweak_order": tweakorder, "use_sextractor": usese, "crpix_center": crpix, "parity": parity})}, files={'file': open(str(dirtosol1file),'rb')})
        if (processprint==True):
            print("\nSUBMISSION:")
            print(R2.json(),"\n")



        r3strm = str(R2.json()["subid"])
        R3 = requests.post(str(r3strs + r3strm))
        if (processprint==True):
            sys.stdout.write("solving ")
        counter = 0

        while len(R3.json()["job_calibrations"])==0:
            if (counter>cutoff):
                print("\nSolution failed, taking over",cutoff,"seconds\nExiting program")
                sys.exit()
            time.sleep(1)
            counter += 1
            if (processprint==True):
                sys.stdout.write(".")
            R3 = requests.post(str(r3strs + r3strm))

        time.sleep(2)
        if (processprint==True):
            print("\n\nSOLVED:")
            print(R3.json())

        r4strm = str(R3.json()["jobs"])[1:-1]
        R4 = requests.post(str(r4strs+r4strm+r4stre))
        rasol2 = R4.json()["ra"]
        decsol2 = R4.json()["dec"]
        orisol2 = R4.json()["orientation"]
        scalesol2 = R4.json()["pixscale"]
        radiussol2 = R4.json()["radius"]
        htsol2 = (header['NAXIS2']*scalesol2)/3600.
        wdsol2 = (header['NAXIS1']*scalesol2)/3600.

        if (processprint==True):
            print("\nMETADATA:")
            print("RA-FITS:",str(rasol2),"deg")
            print("DEC-FITS:",str(decsol2),"deg")
            print("SCANHT:",str(htsol2),"deg")
            print("SCANWD:",str(wdsol2),"deg")
            print("SCANRAD:",str(radiussol2),"deg")
            print("SCANSCAL:",str(scalesol2),"arcsec/px")
            print("SCAN-ORI: Up is",str(orisol2),"degrees E of N")
            print()
            print(str(rasol2),"deg"+","+str(decsol2),"deg"+","+str(htsol2),"deg"+","+str(wdsol2),"deg"+","+str(radiussol2),"deg"+","+str(scalesol2),"arcsec/px"+","+"Up is",str(orisol2),"degrees E of N")
    
        r5str = 'http://nova.astrometry.net/new_fits_file/' + r4strm
        R5 = requests.get(r5str, allow_redirects=True)
        dirtosol2file = dirtofilet[0:-5]+"_sol2"+dirtofilet[-5:]

        r6str = 'http://nova.astrometry.net/corr_file/' + r4strm
        R6 = requests.get(r6str, allow_redirects=True)
        dirtosol2corrfile = self.dirtofile[0:-5]+"_corr"+self.dirtofile[-5:]

        r7str = 'http://nova.astrometry.net/axy_file/' + r4strm
        R7 = requests.get(r7str, allow_redirects=True)
        dirtosol2axyfile = self.dirtofile[0:-5]+"_axy"+self.dirtofile[-5:]

        r8str = 'http://nova.astrometry.net/rdls_file/' + r4strm
        R8 = requests.get(r8str, allow_redirects=True)
        dirtosol2rdlsfile = self.dirtofile[0:-5]+"_rdls"+self.dirtofile[-5:]

        open(str(dirtosol2file), 'wb').write(R5.content)
        if (processprint==True):
            print("\nsecond solution file saved")
        open(str(dirtosol2corrfile), 'wb').write(R6.content)
        if (processprint==True):
            print("corr file saved")
        open(str(dirtosol2axyfile), 'wb').write(R7.content)
        if (processprint==True):        
            print("axy file saved")
        open(str(dirtosol2rdlsfile), 'wb').write(R8.content)
        if (processprint==True):
            print("rdls file saved")



            print("\nASTROMETRY PROGRAM COMPLETE")

In [2]:
file1 = File("C:\\Users\\daniel\\Downloads\\10B-1348_det_imagej.fits")

file1.tworuns("XXXXXXXX",186,9,8,3,8,3,False,True,2,300,False)